### Importing the necessary libraries

Please make sure that, before proceeding further, to have `elasticsearch` docker container running - this is a dependancy that would be used throughout the walkthrough.

In [9]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import json
import warnings

warnings.filterwarnings('ignore')


### Data Loading and Preprocessing

In [2]:
!rm documents.json
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-09-15 21:25:54--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-09-15 21:25:54 (54.7 MB/s) - ‘documents.json’ saved [658332/658332]



In [3]:
# we downloaded the data in the previous cell - here we want to load the JSON object in python by using the json module
# to load it as a python object (in our case its a python dictionary)

with open('documents.json', 'rt') as file:
    docs_raw = json.load(file)

In [4]:
# now to flatten our list of dictionaries - our only preprocessing step

documents = [{'course':course_dict['course'],'section':doc['section'],'question':doc['question'],'text':doc['text']} \
            for course_dict in docs_raw \
            for doc in course_dict['documents']]

documents[0]

{'course': 'data-engineering-zoomcamp',
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."}

### Transform into Embeddings

In [5]:
# Load a pretrained sentence transformer model

model = SentenceTransformer("all-mpnet-base-v2") # best pretrained model in their library

In [6]:
# next we are going to create embeddings on the text field - `or doc` needs to be included in update method
# this ensures that original doc is returned after the update

documents = [doc.update({'text_vector': model.encode(doc['text']).tolist()}) or doc for doc in documents]